# Bridging the Gap: Building an Error-Correcting Model with a Digital Twin

Welcome to one of the most advanced and exciting exercises in our course! So far, you've worked with the `SilicoPumpController` (our fast, *in silico* model) and the physical `PumpBot` (our "ground truth"). You've likely noticed a difference between the colors predicted by the simulation and the colors actually produced by the robot. This is what we call the **"reality gap"**, and it is a fundamental challenge in self-driving labs.

Our *in silico* model is great for rapid testing, but it is an idealized version of reality. The `PumpBot`, on the other hand, reflects the reality (it produces what it produces!) but is much slower to operate.

**The Goal:** What if we could get the best of both worlds? What if we could create a "smarter" digital twin that learns from the `PumpBot`'s data to predict and correct its own errors?

In this notebook, you will build exactly that: an **error-correcting neural network**. This network will learn the difference between the *in silico* model and the `PumpBot`. By combining our original simulation with this new error model, we will create a high-fidelity digital twin that is both fast and accurate. This powerful tool will allow us to find optimal color mixtures in simulation with much greater confidence that they will work on the real hardware.

### The Game Plan

Here is a summary of the steps you'll take to build and test your error-correcting model:

1.  **Gather Ground Truth Data:** Mix a few specific colors using the real `PumpBot` and record the measured RGB values. This data represents reality.
2.  **Calibrate the Digital Twin:** Configure the `SilicoPumpController` so its pure colors match the `PumpBot`'s pure colors. Then, measure the difference in a mixed color between the two systems to see the "reality gap" firsthand.
3.  **Find "Smart" Weights:** We will start with a simple correction method. You will use Bayesian Optimization to find a set of input "weights" `w` that adjusts a recipe for the `SilicoPumpController` to make its output better match the `PumpBot`'s output for a specific mixture. This gives a feel for model calibration.
4.  **Train the Real Corrector Model:** You will use the data you have collected to train a small neural network.
    * **Input to the model:** A color recipe (the volumes to mix).
    * **The model learns to predict:** The *error* or *difference* between the color the `SilicoPumpController` would produce and the color the `PumpBot` would produce for that same recipe.
5.  **Create the Ultimate Digital Twin:** Combine the `SilicoPumpController` and your trained error-corrector model. This new, hybrid model can now predict the `PumpBot`'s behavior with high accuracy.
6.  **Test Your Creation:** Give your new model a target color. It will predict the ideal recipe. You will then run this recipe on the real `PumpBot` and see how close you got!

### Exercise 9.0: Importing Our Tools

As always, we begin by importing the necessary Python packages. Here is a quick rundown of the key players for this exercise:

* **`pump_controller` module**: This gives us all the functions we need to interact with the hardware (`PumpController`) and its digital twin (`SilicoPumpController`), as well as helpful utilities for visualizing results (`visualize_rgb`) and reading past data (`read_logfile`).
* **`Odyssey`**: Our go-to library for Bayesian Optimization. In this notebook, we will use its core components to structure our optimization problems.
* **`torch`**: The core of our machine learning model. PyTorch is a powerful library for creating and training neural networks.
* **Other Packages**: We are also importing `numpy` for numerical operations, `pandas` for data handling, `matplotlib` for plotting, and a few other utilities to make our work smoother.

Let's get our environment set up. Run the following cell to import all the tools we'll need for this exercise.


In [ ]:
# PumpController and SilicoPumpController
from pump_controller import PumpController, get_serial_port, list_serial_ports
from pump_controller import SilicoPumpController
from pump_controller import visualize_rgb, visualize_candidates
from pump_controller import read_logfile

# Odyssey
from odyssey.mission import Mission # Mission
from odyssey.navigators import SingleGP_Navigator # Navigator
from odyssey.navigators.sampler_navigators import Sobol_Navigator # Sampler
from odyssey.navigators import ExpectedImprovement # Acquisition
from odyssey.objective import Objective # Objective

# Other Packages
import torch
import numpy as np
import pandas as pd  
import matplotlib.pyplot as plt  
from IPython import display
from warnings import catch_warnings, simplefilter

### Exercise 9.1: Defining Our Scoring Metric

In any optimization task, we need a way to quantitatively measure success. For our color matching problem, this means we need a function that tells us exactly how "far apart" two colors are. A lower score should mean a better match.

The cell below defines our scoring function, `color_difference`. It calculates the Root Mean Squared Error (RMSE) between the RGB values of two colors. This is a standard way to measure the difference between two sets of values. The closer the score is to 0, the more similar the colors are.

In [ ]:
# Difference between mixed and target colors:
def color_difference(mixed_color, target_color):

    mixed_color = np.array(mixed_color)
    target_color = np.array(target_color)
    # Calculate the sum of root mean squared differences between mixed color and target color
    rmse = np.sqrt(np.mean((mixed_color - target_color)**2, axis=-1))
    return np.sum(rmse)

### Exercise 9.2: Characterizing the PumpBot and Calibrating the Digital Twin

Now we get to the core of the exercise. Our goal is to make our `silicobot` behave as much like the real `pumpbot` as possible. The first step is to teach the `silicobot` what the `pumpbot`'s fundamental colors actually look like.

In this section, you will:
1.  **Connect to the `pumpbot`** and run experiments to measure the "pure" colors (100% Red, 100% Green, etc.) and a baseline mixture of all four. These measurements are our **ground truth**.
2.  Use these ground truth colors to **set the `true_coefficients`** of the `silicobot`. This act of calibration makes our simulation a much better starting approximation of reality.
3.  Directly **compare the `pumpbot` and the newly calibrated `silicobot`** on the same mixture to visualize the "reality gap" we aim to correct.

First, let's connect to the hardware. The following code will initialize the `PumpController`. Make sure your robot is powered on and connected.


In [ ]:
pumpbot = PumpController(ser_port = get_serial_port(), cell_volume = 20.0, drain_time = 20.0, config_file = 'config.json')

Now that the `pumpbot` is connected, we can run the automated experiment. The code below will loop through each of the four primary colors (Red, Green, Blue, and Yellow). In each step of the loop, it will:

1.  Create a mixture recipe for a single pure color (e.g., `[1, 0, 0, 0]` for pure Red).
2.  Command the `pumpbot` to mix and measure that color.
3.  Store the resulting "ground truth" RGB value in a list called `pumpbot_mixed_colors`.

After the loop finishes, the code will plot the four colors you measured, giving you a quick visual confirmation of the data you've collected.

In [ ]:
pumpbot_mixed_colors = []
for i in range(4):
    color_to_mix = [0,0,0,0]
    color_to_mix[i] = 1

    mixed_color = pumpbot.mix_color(color_to_mix)
    
    pumpbot_mixed_colors.append(mixed_color)

# Plot the colors
fig, axs = plt.subplots(1, 4, figsize=(12, 3))  # Create a 1x4 grid of subplots

for i, color in enumerate(pumpbot_mixed_colors):
    # Reshape the color to a 1x1x3 array and normalize to [0, 1]
    color = color.reshape(1, 1, 3) / 255.0

    axs[i].imshow(color)  # Display the color
    axs[i].axis('off')  # Hide the axes

plt.show()

In addition to the pure colors, we need a common reference point to directly compare our `pumpbot` and `silicobot`. For this, we will mix a recipe containing equal parts of all four colors.

This specific mixture will serve as our baseline for comparison. Later, we will ask the `silicobot` to simulate this exact same recipe, allowing us to see the difference, or "reality gap", between the two systems firsthand.

Now, mix equal amounts of each color on pumpbot and visualize this color. We will use the `equal_color_mixture` variable multiple times during the course of this notebook.

In [ ]:
equal_color_mixture = [0.25,0.25,0.25,0.25]
pumpbot_equal_color = pumpbot.mix_color(equal_color_mixture)

visualize_rgb(mixture = equal_color_mixture, 
              rgb = pumpbot_equal_color,
              pump_controller = pumpbot,
              target = None)

print(pumpbot_equal_color)

### Exercise 9.3: Creating and Calibrating the Digital Twin

Now that we have our "ground truth" data from the real `pumpbot`, it is time to create its digital counterpart. In this step, we will initialize our `SilicoPumpController`.

Crucially, immediately after creating it, we will **calibrate** it. We will use the `set_color_coefficient()` method to load the `pumpbot_mixed_colors` we just measured. This tells our simulation the *actual* RGB values of our four base liquids, making it a far more accurate "first-order approximation" of our physical system. This is a critical step in bridging the reality gap.

In [ ]:
silicobot = SilicoPumpController(noise_std = 0)
silicobot.set_color_coefficient(r=pumpbot_mixed_colors[0],
                                g=pumpbot_mixed_colors[1], 
                                b=pumpbot_mixed_colors[2], 
                                y=pumpbot_mixed_colors[3])
print(silicobot.true_coefficients)

Now for the moment of truth. We will use our calibrated `silicobot` to mix the exact same recipe we used on the `pumpbot` earlier (`equal_color_mixture`).

This direct comparison will show us how effective our initial calibration was and give us our first look at the remaining "reality gap" between the simulation's prediction and the real-world result.

In [ ]:
silicobot_equal_color = silicobot.mix_color(equal_color_mixture)

Let's visualize the results of our comparison. The code below will calculate the numerical difference between the `silicobot`'s predicted color and the `pumpbot`'s actual color.

It will then generate a plot showing:
* **The outer ring:** The color recipe we used.
* **The main circle:** The color our calibrated `silicobot` produced.
* **The inner circle:** The "ground truth" color the real `pumpbot` produced.

The numerical `score` and the visual difference you see represent the **reality gap**. This is the error our simple calibrated model still makes. The rest of this notebook is dedicated to building a smart model to predict and eliminate this gap.

In [ ]:
score = color_difference(silicobot_equal_color, pumpbot_equal_color)

visualize_rgb(mixture = equal_color_mixture, 
              rgb = silicobot_equal_color,
              pump_controller = silicobot,
              target = pumpbot_equal_color,
              score = score)

### Exercise 9.4: A First Pass at Correction - Optimizing Input Weights

Before we build a full neural network, let's try a simpler correction method. Our calibrated `silicobot` is a good first approximation, but what if it's systematically over- or under-estimating the strength of certain colors? For example, perhaps the real red ink is slightly weaker than the model assumes, so we need to "turn up" the red input to compensate.

We can try to correct for this by finding a set of four multiplicative "weights" ($w=[w_1, w_2, w_3, w_4]$) that adjusts our input recipes.

**The Optimization Problem:**

* **Goal:** Find the weights `w` that minimize the `color_difference` between the weighted `silicobot` output and the ground truth `pumpbot` color for our baseline `equal_color_mixture`.
* **Method:** We will use Bayesian Optimization via **Odyssey** to intelligently search for the best set of weights. [cite_start]As you learned in the lectures, this is a perfect task for Bayesian Optimization, as we are tuning a few parameters to minimize a "black-box" objective function[cite: 168, 325].

To use Odyssey, we first need to define our objective function in a way it can understand.

The code below creates a function called `find_weights`. This function takes one argument, a list of `weights`, and does the following:
1.  Applies the weights to our baseline `equal_color_mixture`.
2.  Simulates the mixing of this new weighted recipe on the `silicobot`.
3.  Calculates the `color_difference` between the `silicobot`'s output and the real `pumpbot`'s output.
4.  Returns this difference as the score to be minimized.


In [ ]:
def find_weights(weights):
    weighted_silicobot_equal_color = silicobot.mix_color(weights * torch.tensor([0.25, 0.25, 0.25, 0.25]))
    score = color_difference(weighted_silicobot_equal_color, pumpbot_equal_color)
    return score

objective = Objective(find_weights)

With our objective function defined, the next step is to formally define our optimization problem using Odyssey's `Mission` class. A `Mission` object holds all the essential information about our goal.

The code below sets up our mission by specifying:
* `funcs`: A list containing the `objective` function we created in the previous step.
* `maneuvers`: This defines the goal for each objective. Since we want to minimize the `color_difference`, we set this to `['descend']`.
* `envelope`: This defines the search space. It sets the lower and upper bounds for each of the four weights we are optimizing. We are constraining them to be within a sensible range, from a very small positive number up to `1.0`.

In [ ]:
goals = ['descend']

min = 1e-5

param_space = ([min, 1.0], [min, 1.0], [min, 1.0], [min, 1.0])

# Define Mission
mission = Mission(name = 'Silico Error Correcting',
                  funcs = [objective],
                  maneuvers = goals,
                  envelope = param_space)

Now that we have defined our `Mission`, we need to choose an algorithm to carry it out. 

In [ ]:
num_init_design = 5
navigator = SingleGP_Navigator(mission = mission,
                               num_init_design = num_init_design,
                               init_method = Sobol_Navigator(mission = mission),
                               input_scaling = False,
                               data_standardization = False,
                               display_always_max = False,
                               acq_function_type = ExpectedImprovement,
                               acq_function_params = {'best_f': 0.0})

Now we are ready to run the optimization. The code below executes the main Bayesian Optimization loop for a set number of iterations (`num_iter`). As the loop runs, the navigator will progressively find better and better weights to minimize the difference between the `silicobot` and `pumpbot`.

In [ ]:
num_iter = 30
while len(mission.train_X) - num_init_design < num_iter:

    with catch_warnings() as w:
        simplefilter('ignore')
        
        trajectory = navigator.trajectory()
        observation = navigator.probe(trajectory, init = False)

        navigator.relay(trajectory, observation)
        navigator.upgrade()

The optimization is now complete. The `mission` object has logged every set of weights that were tried and the corresponding `color_difference` score.

Let's visualize the performance of our optimizer. The code below will plot the score at each iteration. A successful optimization run should show a clear downward trend, indicating that the `Navigator` was progressively finding better weights that reduced the error between the `silicobot` and `pumpbot`. 

In [ ]:
inputs = mission.display_X
scores = mission.display_Y

fig = plt.figure(figsize=(10, 5))

plt.scatter(range(len(scores)), scores)
plt.title("Weight Optimization Progress")
plt.xlabel("Iteration Number")
plt.ylabel("Color Difference (Score)")
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

The plot shows us the overall trend, but now we need to pinpoint the single best result from our optimization run. The `mission` object contains the history of all tested inputs (`display_X`) and their resulting scores (`display_Y`).

The code below will programmatically find the iteration with the lowest score and retrieve the corresponding set of weights. These are the optimal weights our Bayesian Optimizer discovered to best align the `silicobot` with the `pumpbot` for this specific mixture.

In [ ]:
best_idx = scores.argmin().item()
best_weights = inputs[best_idx]
best_score = scores[best_idx]

print(best_weights)

Now that we have what our optimizer believes are the `best_weights`, let's test them out.

The code below will apply these optimal weights to our original `equal_color_mixture` recipe and run a new simulation on the `silicobot`. This will show us the predicted color from our new, "weight-corrected" digital twin.

In [ ]:
best_weighted_mixed_color = silicobot.mix_color(best_weights * torch.tensor([0.25, 0.25, 0.25, 0.25]))

This is the final check for our weight-tuning experiment. The code below will visualize the new color produced by the `silicobot` using the `best_weights`.

Compare the resulting image and the new `score` to the ones you saw originally. Has the score decreased? Is the color visually a better match?

Hopefully, you will see an improvement! This simple weighting method can be effective for basic calibration. However, it is often too simple to capture the complex, non-linear differences between a simulation and reality. To bridge the gap more effectively, we will need a more powerful tool, which we will build in the next section.

In [ ]:
score = color_difference(best_weighted_mixed_color, pumpbot_equal_color)

print(f'Weights: {torch.round(best_weights, decimals = 3)}')
visualize_rgb(mixture = equal_color_mixture, 
              rgb = best_weighted_mixed_color,
              pump_controller = silicobot,
              target = pumpbot_equal_color,
              score = score)

### Exercise 9.5: Building the Neural Network Error-Correction Model

The weight-based correction was a good first attempt, but it is a linear tool that can't capture the complex, non-linear errors that often occur in physical systems. To truly bridge the reality gap, we need a more powerful and flexible model: a **neural network**.

Our goal is to train a neural network to be an **error-corrector**. Instead of predicting the color directly, this network will learn to predict the *residual error* between our `silicobot` and the `pumpbot`.

**The Machine Learning Task:**
* **Input (X) for our NN:** A color recipe (e.g., `[0.25, 0.25, 0.25, 0.25]`).
* **Output (Y) our NN will predict:** The difference in the resulting RGB values (`pumpbot_color - silicobot_color`).

Once trained, our final high-fidelity digital twin will work like this:
`Corrected_Prediction = silicobot.mix_color(recipe) + NN_error_model.predict(recipe)`

To train this model, we first need data. The following section gives you two options: loading historical data from previous `pumpbot` experiments or generating new data from scratch.

#### Exercise 9.5a: Load Historical Data

The most efficient way to build our training set is to use data from previous experiments. The following cells will guide you through loading previous `.csv` log files from past `pumpbot` runs, cleaning them, and preparing them for training. The more data you have, the better your model will be.

The code below uses the `glob` library to automatically find all files in a directory that end with the `.csv` extension.

**Action Required:** Make sure all your historical `pumpbot` data files are located in a folder named `student_data`. If you have placed them in a different folder, you will need to modify the path in the next code cell accordingly.

In [ ]:
from glob import glob
files = glob('student_data/*.csv') # Get all files in a directory
# files = ['datafile1.csv', 'datafile2.csv', 'datafile3.csv'] # Or specify the files manually

Once the file paths are collected, we need to load and prepare the data for our neural network. The next **data preprocessing** step will:

1.  **Combine Data**: Loop through each `.csv` file path, load it into a pandas DataFrame, and then concatenate them all into a single, large DataFrame containing all your historical data.
2.  **Clean Data**: Remove any rows that might have missing values (`NaN`) and reset the DataFrame's index to keep things tidy.
3.  **Convert Data Types**: This is a crucial step. When saved in a CSV, a list like `[0.1, 0.2, 0.3, 0.4]` is stored as a string (`"[0.1, 0.2, 0.3, 0.4]"`). We cannot perform mathematical operations on strings. The code uses `ast.literal_eval` to safely convert these strings back into Python lists, and then into numerical NumPy arrays that we can use for training.

In [ ]:
import ast

for i in range(len(files)):
    if i == 0:
        df = pd.read_csv(files[i])
    else:
        df = pd.concat([df, pd.read_csv(files[i])])

df = df.dropna() # Remove any nan rows
df = df.reset_index(drop=True) # Reset index

for column in df.columns:
        df[column] = df[column].apply(ast.literal_eval)

# Convert the lists of strings into lists of floats
for column in df.columns:
    df[column] = df[column].apply(lambda x: np.array([float(i) for i in x]))


With our historical data loaded and cleaned, we can now construct the final dataset for training our neural network. This involves creating our input matrix `X` (the features) and our target vector `Y` (the values to be predicted).

**Input `X` (The Recipes):**
The input to our model will be the color mixture recipes. We perform two transformations on the historical recipes from our data:
1.  **Apply Weights:** We multiply the recipes by the `best_weights` we found during our initial calibration.
2.  **Normalize:** We then normalize the weighted recipes so that the volumes in each recipe sum to 1. This is a standard machine learning practice that helps models train more effectively.

**Output `Y` (The Residual Error):**
The target for our model is the **residual error** between the `pumpbot` and the `silicobot`. For each recipe in our dataset, we calculate this error as:

`Y = (Actual pumpbot_color) - (Predicted silicobot_color)`

This `Y` vector of errors is what our neural network will learn to predict.

In [ ]:
# If data is loaded 

X = torch.tensor(df.mixture) # Get the input data from pumpbot results
X = best_weights * X # Scale the points with the best weights
X /= X.sum(dim=1, keepdim=True) # Normalize the points


Y = torch.tensor(df.measurement)
for i in range(len(Y)):
    error = Y[i] - torch.tensor(silicobot.mix_color(X[i]))
    Y[i] = error

#### Exercise 9.5b: Generate New Data (Alternative)

If you do not have sufficient amount of data, you can generate more training set. This section provides an alternative path for data creation.

The process is as follows:
1.  Generate a set of random color recipes.
2.  For each recipe, run an experiment on the **real `pumpbot`** to get the true color.
3.  For the same recipe, run a simulation on the **`silicobot`** to get the predicted color.
4.  Calculate the difference to get the error (`Y`), which will be paired with the recipe (`X`).


In [ ]:
X = torch.rand(60, 4) # Generate random points
X = best_weights * X # Scale the points with the best weights
X /= X.sum(dim=1, keepdim=True) # Normalize the points

Y = []
for i in range(len(X)):
    error = pumpbot.mix_color(X[i]) - silicobot.mix_color(X[i])
    Y.append(error)

Y = torch.stack(Y) 

#### Exercise 9.5c: Prepare Data for Training

Regardless of whether you loaded historical data or generated it new, you now have your complete training dataset (`X` and `Y`). Before we can train our model, we must perform one final, critical step: splitting our data into a **training set** and a **testing set**.

Why do we do this? If we train our model and evaluate it on the same data, we won't know if it has truly learned the underlying patterns or if it has just "memorized" the answers. By setting aside a portion of the data for testing (in this case, 20%), we can get an honest assessment of how well our model performs on data it has never seen before. This is the key to building a model that can generalize.

We will also use PyTorch's `DataLoader` utility, which is a convenient tool that batches and shuffles our data, making the training process more efficient. (Note: This is different from the `DataLoader` in Odyssey).

In [ ]:
from torch.utils.data import DataLoader, random_split, TensorDataset

# Create train, dependent test data
dataset = TensorDataset(X, Y)

total_size = len(dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=train_size, shuffle=True) # Use all data
test_loader = DataLoader(test_dataset, batch_size=test_size, shuffle=False) # Use all data

#### Exercise 9.5d: Define and Tune the Neural Network Model

With our data ready, we can now define and train our error-correction model. This process has two key stages:

**1. Defining a Flexible Model Architecture:**
We will create a general-purpose `MLPRegressor` class. "MLP" stands for Multi-Layer Perceptron, which is a standard type of feed-forward neural network. Our class will be flexible, allowing us to easily create models with different architectures (i.e., a varying number of hidden layers and nodes per layer).

**2. Hyperparameter Tuning with Bayesian Optimization:**
How do we know the best architecture for our model? How many hidden layers should it have? How many nodes in each layer? These choices are called "hyperparameters," and finding the best combination is a difficult optimization problem in itself.

Instead of guessing, we will use **Bayesian Optimization** to solve this! We will set up an Odyssey `Mission` where the goal is to find the neural network architecture that results in the lowest test error. This is a powerful demonstration of using one advanced tool (Bayesian Optimization) to automatically tune another (a Neural Network).

Let's start by defining the architecture for our neural network. The code below creates a flexible `MLPRegressor` class that we can use to build networks of various sizes.  This class takes an input size (in this case 4 nodes - one for each mixture color RGBY), an output size (in this case 3 nodes - one for each measurement color RGB) and a list of hidden layers. As you know, a neural network has a certain number of hidden layers each with a certain amount of nodes, and these numbers can be passed using the `hidden_layers` argument as a list. For example, creating 3 hidden layers with 2, 3 and 4 nodes, respectively:

$$\texttt{hidden\_layers} = [2, 3, 4]$$

The model is constructed from a sequence of layers. Here are the key components you will see in the code:
* **`nn.Linear(in, out)`**: This is a standard fully-connected layer that performs a linear transformation on the data. It's the primary building block of our network.
* **`nn.ReLU()`**: This is our activation function. It introduces non-linearity, allowing the network to learn far more complex patterns than a simple linear model could.
* **`nn.BatchNorm1d`**: Batch Normalization is a standard technique that normalizes the inputs to a layer. This helps to stabilize and accelerate the training process.
* **`nn.Dropout`**: This is a regularization technique to prevent overfitting. During training, it randomly sets a fraction of neuron activations to zero, forcing the network to learn more robust and generalizable features.

In [ ]:
import torch.nn as nn
import torch.optim as optim

class MLPRegressor(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, dropout_prob=0.5):
        super(MLPRegressor, self).__init__()
        self.layers = nn.ModuleList()
        last_size = input_size
        for size in hidden_layers:
            if size > 0:  # Only add the layer if it has more than 0 nodes
                self.layers.append(nn.Linear(last_size, size))
                self.layers.append(nn.BatchNorm1d(size))
                self.layers.append(nn.ReLU())
                self.layers.append(nn.Dropout(dropout_prob))
                last_size = size
        self.layers.append(nn.Linear(last_size, output_size))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

Now we create the objective function for our hyperparameter tuning mission. This function, `train_and_evaluate_model`, will be called by Odyssey. For any given neural network architecture Odyssey wants to test, this function will build it, train it, evaluate it, and report back a single score indicating how well it performed.

Here’s a breakdown of how it works:

* **Input**: The function receives a list of continuous values from Odyssey, representing a proposed architecture (e.g., `[15.6, 25.2, 8.1]`).
* **Output**: It returns a single number: the final test loss (our performance score). A lower score means a better architecture.

**Inside the function, it performs these steps:**
1.  **Define Architecture**: It rounds the continuous input values to the nearest integers (e.g., `[16, 25, 8]`) to define a valid network with a specific number of nodes in each hidden layer. This is a clever workaround to use a continuous optimizer for a discrete problem.
2.  **Build Model**: It creates an instance of our `MLPRegressor` class with this specific architecture.
3.  **Train Model**: It trains the new model on our `train_loader` data for a fixed number of epochs.
4.  **Evaluate Model**: After training, it evaluates the model's performance on the unseen `test_loader` data to get an honest measure of its quality.
5.  **Return Score**: It returns the final test loss, which Odyssey will then use to update its own surrogate model and decide which architecture to try next.

In [ ]:
def train_and_evaluate_model(hidden_layers):
    # Round the number of nodes in each hidden layer to the nearest integer
    hidden_layers = hidden_layers.to(torch.int64).squeeze()

    # Create an instance of the MLP regressor
    model = MLPRegressor(4, 3, hidden_layers)

    model.train()
    # Train the model
    loss_fn = nn.MSELoss() # Mean squared error loss

    optimizer = optim.Adam(model.parameters())
    n_epochs = 100
    for epoch in range(n_epochs):
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            loss = torch.sqrt(loss) # turn mse to rmse loss for each batch
            loss.backward()
            optimizer.step()

    model.eval()
    # Evaluate the model
    total_loss = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            total_loss += torch.sqrt(loss_fn(outputs, targets)) # turn mse to rmse loss for each batch

    return total_loss / len(test_loader) # average rmse over all batches


Now, we formally package our `train_and_evaluate_model` function into an `Objective` object. This is a standard step required by Odyssey to ensure the function is in a format that the `Mission` can use.

In [ ]:
objective = Objective(train_and_evaluate_model)

Now we define the search space for our hyperparameter tuning `Mission`. This `envelope` tells Odyssey the valid range for each parameter it is trying to optimize.

In this case, the parameters we are tuning are the **number of nodes in each potential hidden layer** of our neural network.

The code below sets up a search space for a network with up to 5 hidden layers, where each layer can have between 0 and 20 nodes.
* **`max_n_hidden_layers`**: We set a maximum possible depth for our network.
* **`min_hidden_layer_nodes` / `max_hidden_layer_nodes`**: We set the range for the number of nodes in each of those layers.

This setup allows Odyssey to be very flexible. For example, if it suggests an architecture like `[18, 12, 0, 0, 0]`, our `MLPRegressor` class will interpret this as a two-layer network with 18 and 12 nodes, respectively, since layers with 0 nodes are skipped.

In [ ]:
goals = ['descend']

min_hidden_layer_nodes = 0.0
max_hidden_layer_nodes = 20.0
max_n_hidden_layers = 5

param_space = [[min_hidden_layer_nodes, max_hidden_layer_nodes]] * max_n_hidden_layers

# Define Mission
mission = Mission(name = 'Mix2RGB MLP Tuning',
                  funcs = [objective],
                  maneuvers = goals,
                  envelope = param_space)

Just as we did for the weight optimization, we now need to set up a `Navigator` for our new `Mission`. This `Navigator` will be the "brain" that intelligently searches for the best neural network architecture.

The setup is very similar to before:
* We will use a `SingleGP_Navigator`, which builds a probabilistic model of how different architectures (our input parameters) relate to the model's performance (the test score).
* It will start by trying `num_init_design` (5) initial architectures, chosen by the `Sobol_Navigator` to get an even "first look" at the search space.
* After the initial samples, it will use the `ExpectedImprovement` acquisition function to propose the next most promising architecture to build, train, and test.

In [ ]:
num_init_design = 5
navigator = SingleGP_Navigator(mission = mission,
                               num_init_design = num_init_design,
                               init_method = Sobol_Navigator(mission = mission),
                               input_scaling = False,
                               data_standardization = False,
                               display_always_max = False,
                               acq_function_type = ExpectedImprovement,
                               acq_function_params = {'best_f': 0.0})

We are now ready to launch the automated search for the best neural network architecture. The code below will execute the Bayesian Optimization loop.

Be aware that this process can take some time. In each iteration of the loop, Odyssey will:
1.  **Ask (`navigator.trajectory()`):** Propose a promising new network architecture (e.g., `[18, 25, 10, 5, 0]`).
2.  **Evaluate (`navigator.probe(...)`):** Execute our `train_and_evaluate_model` function. This is the heavy-lifting step that **builds, trains, and tests an entire neural network** based on the proposed architecture.
3.  **Tell (`navigator.relay`, `navigator.upgrade`):** Take the final test score from that network and use it to update its understanding of which architectures perform well, helping it make an even smarter choice in the next iteration.

In [ ]:
num_iter = 50
while len(mission.train_X) - num_init_design < num_iter:

    with catch_warnings() as w:
        simplefilter('ignore')
        
        trajectory = navigator.trajectory()
        observation = navigator.probe(trajectory, init = False)

        navigator.relay(trajectory, observation)
        navigator.upgrade()

Now that the automated search is complete, the `mission` object contains a complete log of every architecture that was tested and its final performance score.

The code below will find the best-performing set of hidden layer sizes, which we will use to build our final error-correction model.

In [ ]:
best_idx = mission.display_Y.argmin()
best_params = mission.display_X[best_idx]
best_metric = mission.display_Y[best_idx]

Let's print the results of our search. The following code will display the best architecture found by Odyssey, showing the number of nodes in each hidden layer, as well as the final test score that this architecture achieved.

In [ ]:
print(best_params.to(torch.int64))
print(best_metric)

Now that we have identified the winning architecture, it's time to create our final, production-ready model.

During the Bayesian Optimization search, we only trained each candidate architecture for a relatively small number of epochs (e.g., 100) to quickly get a performance estimate. Now that we have our single best architecture, we can afford to train it for much longer to ensure it reaches its full potential.

The code below will create one last `MLPRegressor` instance with our `best_params` and train it for 500 epochs. This will be our final, fully-trained error-correction model.

In [ ]:
# Retrain the best model
forward_model = MLPRegressor(4, 3, best_params.to(torch.int64))

forward_model.train()
# Train the model
loss_fn = nn.MSELoss() # Mean squared error loss

optimizer = optim.Adam(forward_model.parameters())
n_epochs = 500
for epoch in range(n_epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = forward_model(inputs)
        loss = loss_fn(outputs, targets)
        loss = torch.sqrt(loss) # turn mse to rmse loss for each batch
        loss.backward()
        optimizer.step()

With our `forward_model` fully trained, we can now use it to make predictions.

Let's test it on our original baseline case: the `equal_color_mixture`. The code below will take this recipe, apply the `best_weights` (to ensure the input format matches what the model was trained on), and then feed it to our `forward_model` to get a prediction for the color error.

Note the use of `.eval()` and `torch.no_grad()`. These are standard PyTorch commands that switch the model from "training mode" to "inference mode," turning off features like dropout and disabling gradient calculations to make predictions faster and more consistent.

In [ ]:
inputs = best_weights * torch.tensor([[0.25, 0.25, 0.25, 0.25]])

forward_model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Temporarily turn off gradient computation
    error_correction = forward_model(inputs)  # Make predictions

Now we put all the pieces together to get our best possible prediction for the baseline color. The code below calculates the final corrected color using our full hybrid model:

**`Final Corrected Color = (Silicobot's prediction for the weighted recipe) + (NN's predicted error)`**

This `Final Corrected Color` represents the output from our new, high-fidelity digital twin. We will then calculate the score between this new prediction and the original `pumpbot` color to see how much of the reality gap we were able to close.

In [ ]:
# Mix a color with silicobot and error correct using the model
# Compare the color with the pumpbot
corrected_equal_color = silicobot.mix_color(best_weights * torch.tensor([0.25, 0.25, 0.25, 0.25])) + error_correction.squeeze().tolist()
score = color_difference(corrected_equal_color,pumpbot_equal_color)

Finally, let's visualize the result from our full, error-corrected digital twin.

The plot below will show the final predicted color. Compare this visualization and its score to our previous attempts:
1.  The uncorrected `silicobot`.
2.  The simple weight-corrected `silicobot`.

Hopefully, you will see a much lower score and a closer visual match, demonstrating the power of using a neural network to capture and correct for the complex errors in our initial simulation.

In [ ]:
visualize_rgb(mixture = equal_color_mixture,
              rgb=corrected_equal_color,
              pump_controller=silicobot,
              target=pumpbot_equal_color,
              score=score)

Phew, that was a lot of work, but look at what you've accomplished! You have successfully:
1.  Quantified the "reality gap" between a simple simulation and a physical robot.
2.  Used Bayesian Optimization to find the best architecture for a neural network.
3.  Trained a neural network to act as a sophisticated error-correction model.
4.  Combined these components to create a **high-fidelity digital twin** that is both fast (like a simulation) and accurate (like the real hardware).

This is a powerful tool. In the final section, we will use this new digital twin to solve a common real-world problem: given a target color, how can we find the recipe to create it?

### Exercise 9.6: Building a Reverse (Inverse) Model to Predict Recipes

So far, we have built an excellent **forward model**: it takes a `recipe` as input and predicts the resulting `color`. This is incredibly useful, but often in science, we face the opposite challenge: the **inverse problem**. We know the outcome we want (a specific `color`), and we need to find the inputs (`recipe`) that will produce it.

Instead of trying to solve this inverse problem with another complex optimization, we can use a powerful machine learning strategy:
1.  **Generate a Large Synthetic Dataset:** We will use our new, high-fidelity digital twin (our `silicobot` + the `forward_model` error corrector) to generate thousands of `(recipe, color)` pairs. This is incredibly fast and cheap compared to running real experiments.
2.  **Train a Reverse Neural Network:** We will then train a *new* neural network, but we'll flip the inputs and outputs from our last model.

**The New Machine Learning Task:**
* **Input (X) for our reverse NN:** An RGB color vector.
* **Output (Y) our reverse NN will predict:** The 4-element recipe required to create that color.

This "reverse model" will act as a powerful tool that can instantly suggest a recipe for any target color we desire.

#### Exercise 9.6a: Generate Data for the Reverse Model

The first step in building our reverse model is to generate a large, high-quality dataset. We will do this by creating a large number of random recipes and using our error-corrected digital twin to predict the resulting color for each one.

Let's begin by creating the **output data** for our reverse model. These are the target values (`Y_reverse`) that our new network will learn to predict.

The code below will:
1.  Generate a large number (`n_points`) of random 4-element recipes.
2.  Apply the `best_weights` we found in the first part of the notebook to these recipes.
3.  Normalize the weighted recipes so that each one sums to 1.0.

This resulting `Y_reverse` tensor will contain a diverse set of valid recipes. In the next step, we will generate the corresponding input colors (`X_reverse`) for each of these recipes.

In [ ]:
n_points = 1000
Y_reverse = best_weights * torch.rand(n_points,4)
Y_reverse /= Y_reverse.sum(dim=1, keepdim=True)

Now that we have the target outputs (`Y_reverse`), we need to generate the corresponding **input data (`X_reverse`)**. These will be the accurately predicted colors for each of our random recipes.

To do this, the code below will loop through every recipe we just created and use our full, high-fidelity digital twin to predict the resulting color. As a reminder, the calculation for each color is:

**`Predicted Color = silicobot.mix_color(recipe) + forward_model.predict(recipe)`**

After this step, we will have our complete, synthetically generated dataset of `(color, recipe)` pairs, ready to train our reverse model.

In [ ]:
X_reverse = []
with torch.no_grad():
    for i in range(len(Y_reverse)):
        x = torch.tensor(silicobot.mix_color(Y_reverse[i])) + forward_model(Y_reverse[i].unsqueeze(0)).squeeze()
        X_reverse.append(x)

X_reverse = torch.stack(X_reverse)

#### Exercise 9.6b: Train the Reverse Model

With our large, synthetic dataset of `(color, recipe)` pairs now complete, we must prepare it for training.

As we did with our first model, we will split the data into a **training set** (for teaching the model) and a **testing set** (for evaluating its performance on unseen data). This is a crucial step to ensure we are building a model that can generalize to new target colors that it wasn't trained on.

We will then load both sets into PyTorch `DataLoader` objects to manage the data efficiently during training.

In [ ]:
# Create train, and test data for reverse model
reverse_dataset = TensorDataset(X_reverse, Y_reverse)

reverse_total_size = len(reverse_dataset)
reverse_train_size = int(0.8 * reverse_total_size)
reverse_test_size = reverse_total_size - reverse_train_size

reverse_train_dataset, reverse_test_dataset = random_split(reverse_dataset, [reverse_train_size, reverse_test_size])

# Dataloaders
reverse_train_loader = DataLoader(reverse_train_dataset, batch_size=reverse_train_size, shuffle=True) # Use all data
reverse_test_loader = DataLoader(reverse_test_dataset, batch_size=reverse_test_size, shuffle=False) # Use all data

Just as we did for our first model, we will now use Bayesian Optimization to find the optimal architecture for our new **reverse model**.

We will create a new objective function, `train_and_evaluate_reverse_model`, that is nearly identical to our previous one. It will again build, train, and test a neural network of a given architecture and return its performance score.

The only crucial difference is that the model's input and output sizes are now flipped to solve the inverse problem:
* **Input Layer Size**: 3 (for an R, G, B color vector)
* **Output Layer Size**: 4 (for the predicted R, G, B, Y recipe)

Odyssey will use this objective function to automatically find the best network for turning a target color into a working recipe.

In [ ]:
def train_and_evaluate_reverse_model(hidden_layers):
    # Round the number of nodes in each hidden layer to the nearest integer
    hidden_layers = hidden_layers.to(torch.int64).squeeze()

    # Create an instance of the MLP regressor
    model = MLPRegressor(3, 4, hidden_layers)

    model.train()
    # Train the model
    loss_fn = nn.MSELoss() # Mean squared error loss

    optimizer = optim.Adam(model.parameters())
    n_epochs = 100
    for epoch in range(n_epochs):
        for inputs, targets in reverse_train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            loss = torch.sqrt(loss) # turn mse to rmse loss for each batch
            loss.backward()
            optimizer.step()

    model.eval()
    # Evaluate the model
    total_loss = 0
    with torch.no_grad():
        for inputs, targets in reverse_test_loader:
            outputs = model(inputs)
            total_loss += torch.sqrt(loss_fn(outputs, targets)) # turn mse to rmse loss for each batch

    return total_loss / len(reverse_test_loader) # average rmse over all batches


As before, we now formally package our `train_and_evaluate_reverse_model` function into an `Objective` object for Odyssey.

In [ ]:
reverse_objective = Objective(train_and_evaluate_reverse_model)

We will now set up the `Mission` for tuning our reverse model. The process is identical to the one we used for the forward model:
* **`envelope`**: We define the same search space, allowing Odyssey to test architectures with up to 5 hidden layers, each having between 0 and 20 nodes.
* **`maneuvers`**: Our goal is still to find the model with the lowest test error, so we will again set our maneuver to `['descend']`.

In [ ]:
goals = ['descend']

min_hidden_layer_nodes = 0.0
max_hidden_layer_nodes = 20.0
max_n_hidden_layers = 5

param_space = [[min_hidden_layer_nodes, max_hidden_layer_nodes]] * max_n_hidden_layers

# Define Mission
mission = Mission(name = 'RGB2Mix MLP Tuning',
                  funcs = [reverse_objective],
                  maneuvers = goals,
                  envelope = param_space)

Now, we set up the `Navigator` that will execute our reverse model tuning `Mission`.

In [ ]:
num_init_design = 5
navigator = SingleGP_Navigator(mission = mission,
                               num_init_design = num_init_design,
                               init_method = Sobol_Navigator(mission = mission),
                               input_scaling = False,
                               data_standardization = False,
                               display_always_max = False,
                               acq_function_type = ExpectedImprovement,
                               acq_function_params = {'best_f': 0.0})

Now, we launch the optimization loop to find the best architecture for our reverse model.

Once again, in each iteration, Odyssey will propose a new network architecture, build it, train it, test it, and use the result to inform its next decision. We are running this search for fewer iterations (`num_iter = 15`) to save time.

In [ ]:
num_iter = 15
while len(mission.train_X) - num_init_design < num_iter:

    with catch_warnings() as w:
        simplefilter('ignore')
        
        trajectory = navigator.trajectory()
        observation = navigator.probe(trajectory, init = False)

        navigator.relay(trajectory, observation)
        navigator.upgrade()

With the search for the best reverse model architecture complete, we will now identify the winning configuration.

The code below will search the `mission` log for the set of hidden layer sizes that resulted in the lowest test error. This is the architecture we will use for our final reverse model.

In [ ]:
best_idx = mission.display_Y.argmin()
best_params = mission.display_X[best_idx]
best_metric = mission.display_Y[best_idx]

Let's display the results of our search for the best reverse model. The following code will print the winning architecture and its final test score.

In [ ]:
print(best_params.to(torch.int64))
print(best_metric)

Now that we have identified the best architecture for our reverse model, it's time to build the final version.

As we did with the forward model, we will now take this winning architecture and train it for a much longer period (500 epochs). This longer training run allows the model's parameters to fully converge, ensuring we get the best possible performance from our optimized architecture. This will be our final `reverse_model`, ready for use.

In [ ]:
# Retrain the best model
reverse_model = MLPRegressor(3, 4, best_params.to(torch.int64))

reverse_model.train()
# Train the model
loss_fn = nn.MSELoss() # Mean squared error loss

optimizer = optim.Adam(reverse_model.parameters())
n_epochs = 500
for epoch in range(n_epochs):
    for inputs, targets in reverse_train_loader:
        optimizer.zero_grad()
        outputs = reverse_model(inputs)
        loss = loss_fn(outputs, targets)
        loss = torch.sqrt(loss) # turn mse to rmse loss for each batch
        loss.backward()
        optimizer.step()

### Exercise 9.7: Putting It All Together - The Final Test

This is the moment of truth. You have built a high-fidelity digital twin and used it to train a powerful reverse model that can predict recipes from colors. Now it's time to test the entire system on the physical `pumpbot`.

The workflow is as follows:
1.  **Get a Target Color:** First, you will need a target color to aim for. You can measure a color from a pre-made solution or simply choose a color you'd like to create.
2.  **Predict the Recipe:** You will feed this target RGB value into your fully-trained `reverse_model`.
3.  **Mix the Predicted Recipe:** Your model will output the ideal recipe. You will then use the `pumpbot` to mix this exact recipe.
4.  **Compare the Result:** Finally, you will measure the color produced by the `pumpbot` and compare it to your original target. This will be the ultimate test of how well your data-driven modeling has bridged the "reality gap."
   
Start by adding a target color to the test_cell and measure the color.

In [ ]:
measured_color = pumpbot.measure()
print(measured_color)

Get the proposed color mixture of this color using the reverse model.

In [ ]:
reverse_model.eval()
with torch.no_grad():
    color_to_mix = reverse_model(measured_color)
    print(color_to_mix)

Use the pumpbot to mix the proposed mixture.

In [ ]:
mixed_color = pumpbot.mix_color(color_to_mix)
print(mixed_color)

Compare the newly mixed color to the target color measurement. Hopefully the colors are identical both visually and in terms of the score!

In [ ]:
visualize_rgb(mixture = color_to_mix.squeeze().tolist(),
              rgb = mixed_color,
              pump_controller=pumpbot,
              target=measured_color.squeeze().tolist(),
              score=color_difference(mixed_color, measured_color))

A lot of work went to getting to this point, and even if you did not get a color that matched the original color, don't worry! You have learned a lot about digital twins, neural network models and using bayesian optimization for tuning hyperparameters and so much more!